In [1]:
import sys
from librosa.core import resample
import pandas as pd
import numpy as np
from IPython.display import Audio
import torch
import pathlib
def create_dir(filename):
    pathlib.Path('/'.join(filename.split('/')[:-1])).mkdir(parents=True, exist_ok=True)
from tqdm.notebook import tqdm
sys.path.append('Conv-TasNet/src/')
sys.path.append('SincNet/')
from conv_tasnet import *
from pit_criterion import cal_loss
from dnn_models import *
from data_io import ReadList,read_conf_inp,str_to_bool
from collections import Counter
import os
device = 3
root = '../'
old_sr = 16000
new_sr = 8000

In [2]:
def load8hz(filename):
    samples = np.load(filename)/(2**15)
    samples = resample(samples, old_sr, new_sr)
    # pad the samples
    if len(samples)>16000:
        samples = samples[:16000]
    if len(samples)<16000:
        padding = np.zeros(16000-len(samples))
        samples = np.concatenate([samples, padding])
    
    return samples

class E2ESet(torch.utils.data.Dataset):
    def __init__(self, root, csv):
        super().__init__()
        self.root = root
        self.csv = pd.read_csv(root+csv)
        self.speakers = list(set(self.csv['first_speaker']))
        self.speakers.sort()
        self.spkr2idx = {spkr:i for i, spkr in enumerate(self.speakers)}
    def __len__(self):
        return len(self.csv)
    def __getitem__(self, idx):
        row = self.csv.iloc[idx]
        sig1, sig2 = load8hz(root+row['first_file']), load8hz(root+row['second_file']) # original files
        spkr1, spkr2 = row['first_speaker'], row['second_speaker']
        target_vec = np.zeros(len(self.speakers))
        target_vec[self.spkr2idx[spkr1]] = 1
        target_vec[self.spkr2idx[spkr2]] = 1
        return sig1+sig2, target_vec
e2eset_train = E2ESet(root, 'overlay-train.csv')
e2eset_val = E2ESet(root, 'overlay-val.csv')
e2eset_test = E2ESet(root, 'overlay-test.csv')

In [3]:
tasnet = ConvTasNet.load_model('final.pth.tar').cuda(device)
if os.path.exists('models/tasnet.pth'):
    print('load tasnet model')
    checkpoint = torch.load('models/tasnet.pth')
    tasnet.load_state_dict(checkpoint['model_state_dict'])
tasnet.train()

    
    
    
    
fs=new_sr
cw_len=200
#cw_shift=10
wlen=int(fs*cw_len/1000.00)

class MixedClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        cnn_arch = {
                'input_dim':wlen,
                'fs':fs,
                'cnn_N_filt':[80,60,60],
                'cnn_len_filt':[251,5,5],
                'cnn_max_pool_len':[3,3,3],
                'cnn_use_laynorm_inp':True,
                'cnn_use_batchnorm_inp':False,
                'cnn_use_laynorm':[True,True,True],
                'cnn_use_batchnorm':[False,False,False],
                'cnn_act':['leaky_relu','leaky_relu','leaky_relu'],
                'cnn_drop':[0.0,0.0,0.0]
                }
        self.cnn_net = SincNet(cnn_arch)

        dnn1_arch = {'input_dim': self.cnn_net.out_dim,
                  'fc_lay': [2048,2048,2048],
                  'fc_drop': [0.0,0.0,0.0], 
                  'fc_use_batchnorm': [True,True,True],
                  'fc_use_laynorm': [False,False,False],
                  'fc_use_laynorm_inp': False,
                  'fc_use_batchnorm_inp': False,
                  'fc_act': ['leaky_relu','leaky_relu','leaky_relu']
                  }
        self.dnn1 = MLP(dnn1_arch)


        dnn2_arch = {'input_dim':2048 ,
                  'fc_lay': [20],
                  'fc_drop': [0.0], 
                  'fc_use_batchnorm': [False],
                  'fc_use_laynorm': [False],
                  'fc_use_laynorm_inp': False,
                  'fc_use_batchnorm_inp': False,
                  'fc_act': ['linear'] # leakyrelu(1) is just identity mapping
                  }
        self.dnn2 = MLP(dnn2_arch)
        
        self.softmax = nn.Softmax(dim = 1)
    def forward(self, X):
        out = self.cnn_net(X)
        out = self.dnn1(out)
        out = self.dnn2(out)
        out = self.softmax(out)
        return out

cls = MixedClassifier().cuda(device)
if os.path.exists('models/sincnet.pth'):
    print('load sincnet model')
    checkpoint = torch.load('models/sincnet.pth')
    cls.load_state_dict(checkpoint['model_state_dict'])
cls.train()


class E2Enet(nn.Module):
    def __init__(self, tasnet, cls):
        super().__init__()
        self.tasnet = tasnet
        self.cls = cls
    def chop_chunk(self, signal):
        batch_size, signal_len = signal.shape
        N_fr=signal_len//wlen
        chunks = []
        for i in range(N_fr):
            chunks.append(signal[..., i*wlen:(i+1)*wlen]) # list of N_fr elements, each (batch_size*wlen)
        return chunks
    def estimate(self, chunks):
        out_vecs = []
        for chunk in chunks:
            out_vecs.append(self.cls(chunk)) # list of N_fr elements, each (batch_size*N_spkr), softmaxed
        out_tensor = torch.stack(out_vecs, dim = 1) # batch_size*N_fr*N_spkr
        out_tensor = out_tensor.mean(dim = 1) # batch_size*N_spkr
        return out_tensor
    def forward(self, sig_mixed):
        sig12 = self.tasnet(sig_mixed) # batch_size*wlen, batch_size*wlen
        sig1, sig2 = sig12[:, 0], sig12[:, 1]
        chunks1, chunks2 = self.chop_chunk(sig1), self.chop_chunk(sig2)
        pred1, pred2 = self.estimate(chunks1), self.estimate(chunks2)
        pred_combined = torch.stack([pred1, pred2], dim = 0) # 2*batch_size*N_spkr
        pred_combined , _ = torch.max(pred_combined, dim = 0) # batch_size*N_spkr
        return pred_combined

e2enet = E2Enet(tasnet, cls).cuda(device)
e2enet.train()
optimizer = torch.optim.Adam(e2enet.parameters(), lr = 0.001)
if os.path.exists('models/e2enet.pth'):
    print('load model')
    checkpoint = torch.load('models/e2enet.pth')
    e2enet.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    loss = checkpoint['loss']
    if 'bestacc' in checkpoint:
        bestacc = checkpoint['bestacc']
    else:
        bestacc = 0.0
else:
    print('using loaded tasnet+sincnet')
    bestacc = 0.0

load tasnet model
load sincnet model
using loaded tasnet+sincnet


In [4]:
def find_max2(tensor):
    array = tensor.cpu().detach().numpy()
    max2 = []
    for row in array:
        max2.append(np.argsort(row)[::-1][:2])
    return np.array(max2)

def compute_corrects(tensor1, tensor2):
    max_1, max_2 = find_max2(tensor1), find_max2(tensor2)
    batch_size = max_1.shape[0]
    batch_corrects = 0
    for i in range(batch_size):
        if Counter(max_1[i])==Counter(max_2[i]):
            batch_corrects+=1
    return batch_corrects

In [5]:
batch_size = 4
e2eloader_test  = torch.utils.data.DataLoader(e2eset_test, batch_size=batch_size, shuffle=True, pin_memory = True, num_workers = 16)
with torch.no_grad():    
    corrects = 0
    for batch_idx, (mixed_sig, target) in enumerate(tqdm(e2eloader_test)):
        mixed_sig, target = mixed_sig.float().cuda(device), target.float().cuda(device)
        out = e2enet(mixed_sig)
        corrects += compute_corrects(out, target)
    print('test acc:', corrects/len(e2eset_test))
pass

Conv-TasNet/src/utils.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  frame = signal.new_tensor(frame).long()  # signal may in GPU or CPU



test acc: 0.8266787658802178


NameError: name 'loss' is not defined